In [ ]:
import duckdb
import os

# Criar conexão em memória
con = duckdb.connect(database=':memory:')

# Instalar e carregar extensões (httpfs é suficiente, aws traz credenciais provider)
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL aws; LOAD aws;")

# Configurar credenciais usando o novo sistema de SECRETS (DuckDB v0.10+)
# Isso garante que o endpoint seja respeitado
con.execute("DROP SECRET IF EXISTS minio_secret;")
con.execute("""
CREATE SECRET minio_secret (
    TYPE S3,
    KEY_ID 'admin',
    SECRET 'password',
    REGION 'us-east-1',
    ENDPOINT 'localhost:9000',
    URL_STYLE 'path',
    USE_SSL 'false'
);
""")

print("DuckDB conectado e configurado para MinIO (via Secrets)!")

## 🔌 Conexão Inicial

In [ ]:
# Instalação do DuckDB e Boto3 (para gerenciamento do MinIO)
!pip install duckdb boto3 pandas -q

## 📦 Instalação e Setup

In [ ]:
import boto3

# Configuração do cliente S3 (MinIO) via Boto3 para criar bucket
s3 = boto3.client('s3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='admin',
    aws_secret_access_key='password',
    region_name='us-east-1'
)

# Criar bucket se não existir
bucket_name = 'datalake'
try:
    if bucket_name not in [b['Name'] for b in s3.list_buckets()['Buckets']]:
        s3.create_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' criado.")
    else:
        print(f"Bucket '{bucket_name}' já existe.")
except Exception as e:
    print(f"Erro ao verificar bucket: {e}")

# Preparar dados: Ler CSV local e escrever Parquet no MinIO
# Nota: Ajuste o caminho do CSV conforme necessário
csv_path = '../sales_data.csv' 
s3_path = f's3://{bucket_name}/sales_data.parquet'

# Verificamos se o arquivo existe localmente
import os
if os.path.exists(csv_path):
    # Usamos DuckDB para converter e enviar (Upload Eficiente)
    con.execute(f"""
        COPY (SELECT * FROM read_csv_auto('{csv_path}')) 
        TO '{s3_path}' (FORMAT 'PARQUET');
    """)
    print(f"Dados enviados para {s3_path}")
else:
    print(f"Arquivo local {csv_path} não encontrado!")

# 📖 Sintaxe Básica de Leitura
print("\nLendo dados do S3:")
con.execute(f"SELECT * FROM '{s3_path}' LIMIT 5;")
print(con.fetchall())

In [ ]:
# Combinar dados locais com dados em S3

# Criar metadados locais (ex: Gerente Regional)
con.execute("""
    CREATE OR REPLACE TABLE gerentes_regionais AS 
    SELECT * FROM (VALUES 
        ('North', 'João Silva'),
        ('South', 'Maria Oliveira'),
        ('East', 'Carlos Santos'),
        ('West', 'Ana Pereira')
    ) AS t(region, manager);
""")

# Join Híbrido: Tabela DuckDB em Memória + Arquivo Parquet no S3
# Usamos a variável 's3_path' definida na célula anterior (s3://datalake/sales_data.parquet)
print("Executando Join Híbrido:")
con.execute(f"""
    SELECT 
        s.region,
        s.sales,
        g.manager
    FROM '{s3_path}' s
    JOIN gerentes_regionais g ON s.region = g.region;
""")
print(con.fetchall())

### Join com Dados Locais

In [ ]:
# Ler de S3, transformar e escrever de volta (ETL)
output_path = f's3://{bucket_name}/processed/regional_totals.parquet'

print(f"Iniciando ETL: {s3_path} -> {output_path}")

con.execute(f"""
COPY (
    SELECT 
        region, 
        sum(sales) as total_sales,
        count(*) as transaction_count
    FROM '{s3_path}'
    GROUP BY region
) TO '{output_path}' (FORMAT 'PARQUET');
""")
print("ETL Concluído com sucesso.")

# Verificar resultado lendo do destino
print("\nDados processados:")
print(con.sql(f"SELECT * FROM '{output_path}'").fetchall())

### ETL Simplificado

In [ ]:
# Analisar logs armazenados em S3 (Simulação de Hive Partitioning)

# 1. Gerar dados de LOG simulados
con.execute("""
    CREATE OR REPLACE TABLE logs_simulados AS
    SELECT 
        range as id,
        CAST('2024-01-01' AS DATE) + (random() * 60)::INT AS log_date,
        CASE WHEN random() > 0.8 THEN 'ERROR' ELSE 'INFO' END as level,
        'Server-' || (random() * 3)::INT as server_name
    FROM range(100);
""")

# 2. Preparar colunas de particionamento (Ano e Mês)
con.execute("""
    CREATE OR REPLACE TABLE logs_partitioned AS
    SELECT 
        *, 
        YEAR(log_date) as year, 
        MONTH(log_date) as month
    FROM logs_simulados
""")

# 3. Escrever particionado no S3 (Hive Style)
logs_path = f's3://{bucket_name}/logs_system'
print(f"Escrevendo logs particionados em: {logs_path} ...")
con.execute(f"""
    COPY logs_partitioned 
    TO '{logs_path}' 
    (FORMAT PARQUET, PARTITION_BY (year, month), OVERWRITE_OR_IGNORE);
""")

# 4. Ler usando Hive Partitioning Automatico
print("\nAnalisando logs diretamente do S3 (Hive Partitioning):")
# O DuckDB detecta o esquema hive automaticamente ao usar glob patterns profundos ou a opção hive_partitioning
con.execute(f"""
    SELECT 
        year, 
        month, 
        level, 
        count(*) as qtd
    FROM '{logs_path}/*/*/*.parquet'
    GROUP BY year, month, level
    ORDER BY year, month, level;
""")
print(con.fetchall())

## 💼 Casos de Uso Práticos

### Análise de Logs

In [ ]:
# Ler múltiplos arquivos filtrando por diretório (Filtragem "Zero-Copy" via nome do arquivo)
# Vamos ler apenas os logs de Janeiro de 2024

pattern_jan = f'{logs_path}/year=2024/month=1/*.parquet'

print(f"Buscando arquivos no padrão: {pattern_jan}")

# DuckDB expande o GLOB e lê apenas os arquivos necessários
con.execute(f"""
    SELECT count(*) as total_jan_logs 
    FROM '{pattern_jan}';
""")

print(con.fetchall())

# Exemplo de lista de arquivos (Glob Expansion)
print("\nArquivos encontrados (primeiros 3):")
files = con.execute(f"SELECT * FROM glob('{pattern_jan}') LIMIT 3").fetchall()
for f in files:
    print(f[0])

## 🔍 Leitura com Glob Patterns

In [ ]:
# Escrever dados para S3 em formato diferente (CSV com Compressão GZIP)
backup_path = f's3://{bucket_name}/backups/logs_backup.csv.gz'

print(f"Exportando backup para: {backup_path}")

con.execute(f"""
    COPY logs_simulados 
    TO '{backup_path}' 
    (FORMAT CSV, COMPRESSION GZIP, HEADER);
""")

print("Backup finalizado com sucesso.")

# Validar a leitura do arquivo comprimido diretamente do S3
print("\nValidando backup (Count):")
res = con.sql(f"SELECT count(*) FROM '{backup_path}'").fetchall()
print(f"Linhas no backup: {res[0][0]}")